In [ ]:
import pandas as pd
import numpy as np
import string

def make_visit(visit_number,target_num,target_name,onband_filter_num,offband_filter_num,
               onband_time=400, offband_time=200):
   visit = f"""\n\n\n      Visit_Number: {visit_number}
         Visit_Label: visit target number {target_num}
   Visit_Requirements: 
   On_Hold_Comments: 
      Visit_Comments: 

         Exposure_Number: 1
         Exposure_Label: 
            Target_Name: {target_name}
                  Config: WFC3/UVIS
                  Opmode: ACCUM
               Aperture: UVIS2-C512C-SUB
            Sp_Element: F{onband_filter_num}M
            Wavelength: 
   Optional_Parameters: FLASH=12
   Number_of_Iterations: 1
         ETC_Run_Number: 
      Time_Per_Exposure: {onband_time} S
   Special_Requirements: 
            PATTERN 1 1-2
         Sub_Exposures: (
      (set-exposure-properties :exposure-number 1 :ppos 1
         :cpos nil :copy nil :split nil :scan nil :orbit-number 1 :duration nil) 
      (set-exposure-properties :exposure-number 1 :ppos 2
         :cpos nil :copy nil :split nil :scan nil :orbit-number 1 :duration nil) 
      (set-exposure-properties :exposure-number 1 :ppos 3
         :cpos nil :copy nil :split nil :scan nil :orbit-number 1 :duration nil) )
               Comments: 

         Exposure_Number: 2
         Exposure_Label: 
            Target_Name: {target_name}
                  Config: WFC3/UVIS
                  Opmode: ACCUM
               Aperture: UVIS2-C512C-SUB
            Sp_Element: F{offband_filter_num}M
            Wavelength: 
   Optional_Parameters: FLASH=14
   Number_of_Iterations: 1
         ETC_Run_Number: 
      Time_Per_Exposure: {offband_time} S
   Special_Requirements: 
         Sub_Exposures: (
      (set-exposure-properties :exposure-number 2 :ppos 1
         :cpos nil :copy nil :split nil :scan nil :orbit-number 1 :duration nil) 
      (set-exposure-properties :exposure-number 2 :ppos 2
         :cpos nil :copy nil :split nil :scan nil :orbit-number 1 :duration nil) 
      (set-exposure-properties :exposure-number 2 :ppos 3
         :cpos nil :copy nil :split nil :scan nil :orbit-number 1 :duration nil) )
               Comments: 
   """
   return visit

# read target df and original .prop with no visit entries
snap = pd.read_csv("/home/insepien/research-data/alpaka/snap/snap146.csv")
with open("/home/insepien/research-data/alpaka/snap/146tar_novisit.prop", "r") as f:
    d = f.readlines()

# add filters to df
flt = []
for z in snap['Z']:
    if 0.1 < z < 0.14:
        filter = '547-621'
    elif 0.14 < z < 0.2:
        filter = '763-689'
    elif 0.2 < z < 0.25:
        filter = '621-689'
    elif 0.25 < z < 0.3:
        filter = '621-763'
    else:
        filter = '689-763'
    flt.append(filter)

snap['flt'] = flt
# check filters
# fs = list(set(snap['flt'].values))
# [print(f"F{i}M -- min z: {snap[snap['flt'] == i]['Z'].min():.3f}, max z: {snap[snap['flt'] == i]['Z'].max():.3f}") for i in fs]


In [ ]:
# make visit numbers. Each visit number can only have 2 characters, so >100 is reformatted to 0A,0B...1A,1B...
letters = string.ascii_uppercase
moreVnumbers = [f'0{let}' for let in letters] + [f'1{let}' for let in letters]
allVnumbers = list(range(100))[1:] + moreVnumbers
# make visits for each row in df
visits = []
for i in range(len(snap)):
    visit_number = allVnumbers[i]
    # visit label = target number in APT
    visit_label = i+1
    target_name = snap.loc[i,'Name']
    onband_filter_num = snap.loc[i,'flt'][:3]
    offband_filter_num = snap.loc[i,'flt'][-3:]
    visit = make_visit(visit_number,visit_label,target_name,onband_filter_num,offband_filter_num)
    visits.append(visit)

# add visits and write to .prop file
new_prop = d.copy()
new_prop += visits
# with open("/home/insepien/research-data/alpaka/snap/full_proposal.prop", "w") as f:
#     f.write("".join(new_prop))